<a href="https://colab.research.google.com/github/FreeworkEarth/Automated-custom-Scaled-YOLOv4-object-detection-model-training-for-total-beginners-/blob/main/01_YOLO_annotated_Dataset_rearrange_SCALED_YOLO_datastructure.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
############
# requirements
!pip install scikit-learn

# packages
import numpy as np
from sklearn.model_selection import train_test_split
import os
import pathlib
from pathlib import Path, PosixPath, WindowsPath, PurePath, PureWindowsPath, PurePosixPath
import shutil
import glob
import subprocess 

import zipfile
import datetime
import random

ROOT_DIR = os.getcwd()
print(ROOT_DIR)

now = datetime.datetime.now()
print ("Current date and time : ")
print (now.strftime("%Y-%m-%d %H:%M:%S"))

/content
Current date and time : 
2021-08-02 15:22:40


# Parameters

### Mandatory


In [ ]:
path_to_labelled_raw_dataset = '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Stop sign_Traffic sign_Car_Street light_Human body_Human face'
#path_to_labelled_raw_dataset = 'YOLO/DATASET/Vehicle registration plate_Stop sign_Traffic sign_Car_Street light_Human body_Human face'
#path_to_labelled_raw_dataset = '/YOLO/DATASET/Vehicle registration plate_Stop sign_Traffic sign_Car_Street light_Human body_Human face'
path_to_labelled_raw_dataset =  'Google Drive\YOLO\DATASET\Vehicle registration plate_Stop sign_Traffic sign_Car_Street light_Human body_Human face'
path_to_labelled_raw_dataset =   'Google Drive\YOLO\Polymer_Anomaly_det_04_21\Dataset'
path_to_labelled_raw_dataset =  '/content/gdrive/MyDrive/YOLO/DATASET/Traffic sign_Car_Street light_Human body'

#@title MANDATORY PARAMETERS FORM 
#@markdown DATASET
path_to_labelled_raw_dataset =  '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21' #@param {type: 'string'}

### Not Mandatory

In [ ]:
#@title NON - MANDATORY PARAMETERS FORM 

#@markdown Percent Training images from full dataset (~60-75%)
train_percent = 0.7 #@param {type: "slider", min:0.6 , max: 0.75, step:0.01}

#@markdown Percent Validation images from full dataset (~10-20%)
valid_percent = 0.15 #@param {type: "slider", min:0.1 , max: 0.2, step:0.01}

#@markdown .zip filename final dataset
zipfile_full_dataset = 'scaledYOLO_Dataset' #@param {type: "string"}

In [ ]:
test_percent = 1 - train_percent - valid_percent
print(train_percent, test_percent, valid_percent)
use_gdrive = 1
# unused so far 
YOLO_models_dataset = ['Darknet', 'Scaled YOLO', 'TensorFlow']

0.7 0.15000000000000005 0.15


In [ ]:
# params not change for fully automated training (in next step)
test_name = 'test'
train_name = 'train'
valid_name = 'valid'
images_name = 'images'
labels_name = 'labels'

In [ ]:
if use_gdrive==1: 
  ### MOUNT Google Drive
  from google.colab import drive
  drive.mount('/{}/gdrive'.format(ROOT_DIR))
  !ln -s /content/gdrive/My\ Drive/ /mydrive
  !ls /mydrive
else:
  pass

Mounted at //content/gdrive
 000_Medium
 00_GAIA
 00_Masterthesis
'01_masterthesis_latex (38).docx'
'01_masterthesis_latex (6).pdf'
'02072019_LS_VarReg (1).gdoc'
'02072019_LS_VarReg (2).gdoc'
 02072019_LS_VarReg.gdoc
'05082019_LS_FCB Hertha.gdoc'
'06102019_CH_FCH _VFL BO (1).gdoc'
'06102019_CH_FCH _VFL BO.gdoc'
'09112019_CH_FCB _BVB.gdoc'
'1019538758(1).csv'
 1019538758.csv
'~$102019_CH_FCH _VFL BO (1).gdoc'
'~$102019_CH_FCH _VFL BO (2).gdoc'
'~$102019_CH_FCH _VFL BO.gdoc'
'2015-02-08 11.52.19.jpg'
 20180327_Siemens_Airport_James.pptx
 2019_Book_ThePythonWorkbook.pdf
 20210209-8067977-umsatz.CSV
 7_tage_inzidenz_03022021.pdf
 AI_ML
 amazon_rücksendung_Hemden_tracht.PNG
 Antrag-auf-Verlängerung-der-Frist-zur-Ablegung-der-GOP_Studienzeitverlängerung_TechFak.pdf
 Antragsformular_Anerkennung_WING-MB_BA.docx
 AttendeeGuide_QiskitGlobalSummerSchoolStudent.pdf
 Bewerbung_Jobs
'Bewerbungsinfomrationen Consulting.docx'
 Bilder
 Bildungskredt_Master.pdf
'bus 60.pdf'
 CGI_Design
 CICD_utils_CL

## Presettings

In [ ]:
## automated path corrections (correct common mistakes automatically)
print(path_to_labelled_raw_dataset)

if path_to_labelled_raw_dataset.startswith('\Google Drive'):
   path_to_labelled_raw_dataset = path_to_labelled_raw_dataset.replace('\Google Drive', '/')
elif path_to_labelled_raw_dataset.startswith('Google Drive'):
     path_to_labelled_raw_dataset = path_to_labelled_raw_dataset.replace('Google Drive', '/')
else:
  pass

print(path_to_labelled_raw_dataset)
if not path_to_labelled_raw_dataset.startswith('/') or path_to_labelled_raw_dataset.startswith('\\'):
  path_to_labelled_raw_dataset =  '/' +  path_to_labelled_raw_dataset
else:
  pass

print(path_to_labelled_raw_dataset)

path_to_labelled_raw_dataset = path_to_labelled_raw_dataset.replace('\\', '/')
print(path_to_labelled_raw_dataset)

/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21


In [ ]:
## Directory 
if use_gdrive == 1:
  ROOT_DIR = '/content'
  print(ROOT_DIR)
  if path_to_labelled_raw_dataset.startswith('/content'):
    pass
  else:
    path_to_labelled_raw_dataset = ROOT_DIR + '/gdrive/MyDrive' +  path_to_labelled_raw_dataset
  print(path_to_labelled_raw_dataset)

/content
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21


In [ ]:
os.chdir(path_to_labelled_raw_dataset)
print(path_to_labelled_raw_dataset)
#print(len(path_to_labelled_raw_dataset))
print(sorted(os.listdir()))
print(len(sorted(os.listdir())))
!ls

/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21
['000f269d7fece350.jpg', '000f269d7fece350.txt', '002bdd2550665a64.jpg', '002bdd2550665a64.txt', '002d099996f3f443.jpg', '002d099996f3f443.txt', '002f860e692757f7.jpg', '002f860e692757f7.txt', '0030164dc0dba5d9.jpg', '0030164dc0dba5d9.txt', '0030feaa83a02c0d.jpg', '0030feaa83a02c0d.txt', '0039ac4bfb8bd69d.jpg', '0039ac4bfb8bd69d.txt', '00494d0811b9e652.jpg', '00494d0811b9e652.txt', '004cc310bb32da6a.jpg', '004cc310bb32da6a.txt', '005932632ad92371.jpg', '005932632ad92371.txt', '00670d2aef4f1fcf.jpg', '00670d2aef4f1fcf.txt', '00765d32b21fe7a4.jpg', '00765d32b21fe7a4.txt', '009dc03a884f0bae.jpg', '009dc03a884f0bae.txt', '00a09b822d470896.jpg', '00a09b822d470896.txt', '00b5ac570d995133.jpg', '00b5ac570d995133.txt', '00c462941507d432.jpg', '00c462941507d432.txt', '00cbc1358d338fe0.jpg', '00cbc1358d338fe0.txt', '0112fff372a9199f.jpg', '0112fff372a9199f.txt', '01244840c068d87a.jpg', '01244840c

In [ ]:
def dataset_files_correct(path_to_labelled_raw_dataset):
  file_dict = {}
  list_all_dataset_images = [] #images and textfiles
  image_string_start_list = []  
  labels_file_list = []


  for file in os.listdir(path_to_labelled_raw_dataset):
          if os.path.isfile(os.path.join(path_to_labelled_raw_dataset, file)):
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png'):
              imagefile = file
              list_all_dataset_images.append(imagefile)
              image_string_start = imagefile.split('.')
              image_string_start_list.append(image_string_start)
              print(len(image_string_start_list))

              ## create textstring from imagefile and check if textfile in directory
              string_textfile = image_string_start[0] + '.txt'
              for file in os.listdir(path_to_labelled_raw_dataset):
                if os.path.isfile(os.path.join(path_to_labelled_raw_dataset, file)):
                  if file == string_textfile:   # image string = 
                    print(file)
                    labels_file_list.append(file)
                    len(labels_file_list)
                  else:
                    pass
                    #print('Image File without label = {}'.format(imagefile))

   ## check if lists are the same length          
  if len(list_all_dataset_images) == len(labels_file_list):
    print(list_all_dataset_images)
    print(labels_file_list)
    print('DATASET complete, all images have corresponding label text files')
  else:
    print(list_all_dataset_images)
    print(labels_file_list)
    print('ERROR-------------DATASET damaged/incomplete----- add label files')
    raise Exception('ERROR-------------DATASET damaged/incomplete----- add label files') 



In [ ]:
check_dataset = dataset_files_correct(path_to_labelled_raw_dataset)

1
000f269d7fece350.txt
2
002bdd2550665a64.txt
3
002d099996f3f443.txt
4
002f860e692757f7.txt
5
0030164dc0dba5d9.txt
6
0030feaa83a02c0d.txt
7
0039ac4bfb8bd69d.txt
8
00494d0811b9e652.txt
9
004cc310bb32da6a.txt
10
005932632ad92371.txt
11
00670d2aef4f1fcf.txt
12
00765d32b21fe7a4.txt
13
009dc03a884f0bae.txt
14
00a09b822d470896.txt
15
00b5ac570d995133.txt
16
00c462941507d432.txt
17
00cbc1358d338fe0.txt
18
0112fff372a9199f.txt
19
01244840c068d87a.txt
20
01ce2560aff03477.txt
21
021a3f4ad16708b8.txt
22
027348743a9b8374.txt
23
02a01a62fa5a9831.txt
24
02b15a54068319fc.txt
25
030b9726b02097ee.txt
26
033ecc2e897a1ebf.txt
27
0354ed1629a4b1b2.txt
28
03d18376979a3104.txt
29
03d1b4e7ad05beae.txt
30
03fcc4d4897ce9a0.txt
31
0448b2215866ada5.txt
32
047f55a198882d7c.txt
33
04c856a70bb52457.txt
34
04ea4e67d46cd424.txt
35
0500f85f6c7f191d.txt
36
05355bdf2b20ddb1.txt
37
056a66236e64de0c.txt
38
05c52ccdb095f060.txt
39
0647c745934ff80e.txt
40
06501a5ed994c937.txt
41
06861fda7b81fa48.txt
42
06937ef97333f924.txt
4

# Get Images and Labelfiles(.txt) from origin dataset

In [ ]:
def get_images_and_labels_from_dir(path_to_labelled_raw_dataset):
  
  list_all_dataset_files = [] #images and textfiles
  for file in os.listdir(path_to_labelled_raw_dataset):
          if os.path.isfile(os.path.join(path_to_labelled_raw_dataset, file)):
            if file.endswith('.jpg') or file.endswith('.jpeg') or file.endswith('.png') or file.endswith('.txt'):
              #print(file)
              list_all_dataset_files.append(file)

  print(list_all_dataset_files)
  print('length complete dataset: {}'.format(len(list_all_dataset_files)))
  if 'classes.txt' in list_all_dataset_files:
    path_classes_txt = os.getcwd()
    print('classes.txt in folder')
    classes_pos = list_all_dataset_files.index('classes.txt')
    print('Postition classes txt: {}'.format(classes_pos))
    list_all_dataset_files.remove('classes.txt')

  # remove classes.txt from list to only have raw dataset
  #for i in range(len(list_all_dataset_files)):
    #if list_all_dataset_files[i].endswith('classes.txt'):
      #list_all_dataset_files.remove('classes.txt')
      #list_all_dataset_files.remove(list_all_dataset_files[i])
      
  print(list_all_dataset_files)
  sorted_dataset_files_list = sorted(list_all_dataset_files)
  print(sorted_dataset_files_list)
  numbers_origin_dataset_files = len(sorted_dataset_files_list)
  print('Number of files origin Dataset(only img + txt without classes txt): {}'.format(numbers_origin_dataset_files))

  return sorted_dataset_files_list, numbers_origin_dataset_files, path_classes_txt

In [ ]:
## call function get_images_and_labels_from_dir
images_and_label_filelist = get_images_and_labels_from_dir(path_to_labelled_raw_dataset)
sorted_dataset_files_list = images_and_label_filelist[0]
numbers_origin_dataset_files = images_and_label_filelist[1]
path_classes_txt = images_and_label_filelist[2]
print(path_classes_txt)

['92fe1e40bd2b2b76.txt', '4673da8c7d02d3ab.txt', 'b3aa29cc695d8bed.txt', '165cdd90a424ed9c.txt', '03fcc4d4897ce9a0.txt', '341939452cfc0e90.txt', 'b3f3fb5f71e6c843.txt', '7a1421752324435c.txt', 'd2e4c0911fcbcae7.txt', '0a623bc78056163d.txt', 'c01b665befdb2139.txt', 'a1f9ea596c18f642.txt', '1b36b89cbb47f5ec.txt', '46fea07f331200f5.txt', '06d93beafd6437f2.txt', '6cd5af0fbddefdd4.txt', '01ce2560aff03477.txt', '580846c6b3b1c622.txt', '5fd977f4db5858a4.txt', '72a50835c7daacaa.txt', '79f809c1004fbb85.txt', 'd52f4fd63a9bbdd8.txt', 'ec12f9374ef7bb69.txt', 'ae2f07097f102a33.txt', '252b3a22b517aae9.txt', '62461632de2f4f40.txt', '15df6acf14b2c1fd.txt', '56bb6488b8aaa4b6.txt', '568787dca9549720.txt', '2f6e61d9c1a9fe68.txt', '892601232994ed3b.txt', 'b2219b84e47bca06.txt', '6880f8bba8f70d55.txt', '53a9786e039ab430.txt', 'ef45c9495fedd219.txt', '118cf7b87ad911a3.txt', '846a837ffec1f3d1.txt', 'aa91332e81496c1e.txt', 'b10cc4d196caf4a4.txt', 'b04254ed4c479cfa.txt', 'badcd10c95dd2eea.txt', 'c6161c0eeabc82

# Calculate amount of data for each part (train, test, valid) that no files are left or to much due to rounding


In [ ]:
## get number total files (=numbers_origin_dataset_files) and rounded number of imagefiles (train,valid,test),
## check if left files after rounding is 0 to continue (no list out of bound later)
print('Number of files origin Dataset: {}'.format(numbers_origin_dataset_files))
print('Number of images origin Dataset: {}'.format(numbers_origin_dataset_files/2))


number_imagefiles_in_data =  len(sorted_dataset_files_list)/2
number_testfiles  = test_percent * number_imagefiles_in_data
number_trainfiles = train_percent * number_imagefiles_in_data
number_validfiles = valid_percent * number_imagefiles_in_data
rounded_number_testfiles  = round(number_testfiles)
rounded_number_trainfiles = round(number_trainfiles)
rounded_number_validfiles = round(number_validfiles)
full_amount_file_after_round = rounded_number_testfiles + rounded_number_trainfiles + rounded_number_validfiles
left_files_amount = len(sorted_dataset_files_list)/2 - rounded_number_testfiles - rounded_number_trainfiles - rounded_number_validfiles

print(number_testfiles)
print(number_trainfiles)
print(number_validfiles)
print('Rounded number of testfiles:{}'.format(rounded_number_testfiles))
print('Rounded number of training files:{}'.format(rounded_number_trainfiles))
print('Rounded number of validation files:{}'.format(rounded_number_validfiles))
print('Full_amount_images_after_round:{}'.format(full_amount_file_after_round*2))
print('left image files before correction: {}'.format(left_files_amount))
print('number dataset files before correction: {}'.format(numbers_origin_dataset_files))

if left_files_amount == -1:
    #numbers_origin_dataset_files = numbers_origin_dataset_files - 2
    rounded_number_testfiles = rounded_number_testfiles - 1
    left_files_amount = len(sorted_dataset_files_list)/2 - rounded_number_testfiles - rounded_number_trainfiles - rounded_number_validfiles
elif left_files_amount == 1:
    #numbers_origin_dataset_files = numbers_origin_dataset_files + 2
    rounded_number_testfiles = rounded_number_testfiles + 1
    left_files_amount = len(sorted_dataset_files_list)/2 - rounded_number_testfiles - rounded_number_trainfiles - rounded_number_validfiles
else:
    pass

print('left files after correction: {}'.format(left_files_amount))
print('number testfiles images after correction: {}'.format( rounded_number_testfiles))
print('number dataset files after correction: {}'.format(numbers_origin_dataset_files))


Number of files origin Dataset: 1176
Number of images origin Dataset: 588.0
88.20000000000003
411.59999999999997
88.2
Rounded number of testfiles:88
Rounded number of training files:412
Rounded number of validation files:88
Full_amount_images_after_round:1176
left image files before correction: 0.0
number dataset files before correction: 1176
left files after correction: 0.0
number testfiles images after correction: 88
number dataset files after correction: 1176


# CREATE Folder structure scaled-YOLO

In [ ]:
## create folder structure for Torch implemented Scaled YOLO
# create_scaledYOLO_folder_structure(path_labelled_dataset, name_mainfolder= (default =train,test,valid), name_imagefolder (default= images), name_labelsfolder (default = labels)):

def create_scaledYOLO_folder_structure(path_labelled_dataset, name_mainfolder, name_imagefolder, name_labelsfolder):

  foldernames_dict = locals()
  print(foldernames_dict)
  print(type(foldernames_dict))
  print(foldernames_dict.items())
  print(foldernames_dict.keys())
  print(foldernames_dict.values())

  folder_test = []
  
  for key,value in foldernames_dict.items():
    if value == str(path_labelled_dataset):
        folder_test.append(value)
    elif value == str(name_mainfolder):  
      folder_test.append(folder_test[0] + '/' + value)
    else:
      folder_test.append(folder_test[0] + '/' + name_mainfolder + '/' + value)



  path_main_folder    = path_labelled_dataset + '/' + name_mainfolder
  path_images_folder  = path_main_folder + '/' + name_imagefolder 
  path_labels_folder  = path_main_folder + '/' + name_labelsfolder
  print(path_main_folder)
  print(path_images_folder)
  print(path_labels_folder)


  if not os.path.exists(path_main_folder):
      os.makedirs(path_main_folder)
      print('Not Dir but created')
  else:
    print('DIR exists')
  if not os.path.exists(path_images_folder):
      os.makedirs(path_images_folder)
      print('test images Not Dir but created')
  else:
    print('DIR test Images exists')
  if not os.path.exists(path_labels_folder):
      os.makedirs(path_labels_folder)
      print('test Labels Not Dir but created')
  else:
    print('DIR test Labels exists')


  print(folder_test)
  return folder_test


In [ ]:
## create folderstructures for Scaled YOLOv4:arguments = (path_dataset, test, images, labels)
#create_scaledYOLO_folder_structure(path_to_labelled_raw_dataset, 'test', 'images', 'labels')

#test_folder_list = create_scaledYOLO_folder_structure(path_to_labelled_raw_dataset, test_name , images_name, labels_name)

#print('testfolder list:{}'.format(test_folder_list))
#test_folder = test_folder_list[1]
#print('test_foldername{}'.format(test_folder))
#path_test_images = test_folder_list[2]
#print(path_test_images)
#path_test_labels = test_folder_list[3]
#print(path_test_labels)



test_folder_list = create_scaledYOLO_folder_structure(path_to_labelled_raw_dataset, test_name, images_name, labels_name)
print(test_folder_list)

test_folder = test_folder_list[1]
print(test_folder)
test_folder_splitted = test_folder.split('/')
name_test =  test_folder_splitted[-1]
print(name_test)

path_test_images = test_folder_list[2]
print(path_test_images)
test_image_folder_splitted = path_test_images.split('/')
name_test_images =  test_image_folder_splitted[-1]
print(name_test_images)

path_test_labels = test_folder_list[3]
print(path_test_labels)

{'path_labelled_dataset': '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21', 'name_mainfolder': 'test', 'name_imagefolder': 'images', 'name_labelsfolder': 'labels'}
<class 'dict'>
dict_items([('path_labelled_dataset', '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21'), ('name_mainfolder', 'test'), ('name_imagefolder', 'images'), ('name_labelsfolder', 'labels')])
dict_keys(['path_labelled_dataset', 'name_mainfolder', 'name_imagefolder', 'name_labelsfolder'])
dict_values(['/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21', 'test', 'images', 'labels'])
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/test
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/test/images
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Tra

In [ ]:
#Train folder structure
train_folder_list = create_scaledYOLO_folder_structure(path_to_labelled_raw_dataset, train_name, images_name, labels_name)
print(train_folder_list)

train_folder = train_folder_list[1]
print(train_folder)
train_folder_splitted = train_folder.split('/')
name_train =  train_folder_splitted[-1]
print(name_train)

path_train_images = train_folder_list[2]
print(path_train_images)
train_image_folder_splitted = path_train_images.split('/')
name_train_images =  train_image_folder_splitted[-1]
print(name_train_images)

path_train_labels = train_folder_list[3]
print(path_train_labels)

{'path_labelled_dataset': '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21', 'name_mainfolder': 'train', 'name_imagefolder': 'images', 'name_labelsfolder': 'labels'}
<class 'dict'>
dict_items([('path_labelled_dataset', '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21'), ('name_mainfolder', 'train'), ('name_imagefolder', 'images'), ('name_labelsfolder', 'labels')])
dict_keys(['path_labelled_dataset', 'name_mainfolder', 'name_imagefolder', 'name_labelsfolder'])
dict_values(['/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21', 'train', 'images', 'labels'])
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/train
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/train/images
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plat

In [ ]:
#Valid folder structure
valid_folder_list = create_scaledYOLO_folder_structure(path_to_labelled_raw_dataset, valid_name, images_name, labels_name)
print(valid_folder_list)

valid_folder = valid_folder_list[1]
print(valid_folder)
valid_folder_splitted = valid_folder.split('/')
name_valid =  valid_folder_splitted[-1]
print(name_valid)

path_valid_images = valid_folder_list[2]
print(path_valid_images)
valid_image_folder_splitted = path_valid_images.split('/')
name_valid_images =  valid_image_folder_splitted[-1]
print(name_valid_images)

path_valid_labels = valid_folder_list[3]
print(path_valid_labels)

{'path_labelled_dataset': '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21', 'name_mainfolder': 'valid', 'name_imagefolder': 'images', 'name_labelsfolder': 'labels'}
<class 'dict'>
dict_items([('path_labelled_dataset', '/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21'), ('name_mainfolder', 'valid'), ('name_imagefolder', 'images'), ('name_labelsfolder', 'labels')])
dict_keys(['path_labelled_dataset', 'name_mainfolder', 'name_imagefolder', 'name_labelsfolder'])
dict_values(['/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21', 'valid', 'images', 'labels'])
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/valid
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/valid/images
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plat

# Donwload raw dataset (YOLO labelled + classes txt) from Google Cloud Bucket or ROBOFLOW

In [ ]:
#os.chdir(ROOT_DIR)
#!curl -L "https://public.roboflow.com/ds/W1iRBYSRW7?key=OivIoqkYvE" > roboflow.zip; unzip roboflow.zip; rm roboflow.zipy

# Get Classes from classes.txt and create data.yaml configuration

In [ ]:
## get content of classes.txt from dataset as list to create config for YOLO (data.yaml for scaled YOLO
# input path to dataset , name of classes.txt
# output: content of classes.txt as list
def get_classes_list_from_classes_txt(path_to_labelled_raw_dataset, name_classes_txt):
  print(path_to_labelled_raw_dataset)
  os.chdir(path_to_labelled_raw_dataset)
  with open(name_classes_txt) as f:
    content = f.readlines()
    # you may also want to remove whitespace characters like `\n` at the end of each line
    content = [x.strip() for x in content]

  return content 

In [ ]:
print(path_to_labelled_raw_dataset)

/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21


In [ ]:
## Create classes list from classes txt to create config for YOLO (data.yaml for scaled YOLO)
classes_txt_list = get_classes_list_from_classes_txt(path_classes_txt, 'classes.txt')
print(classes_txt_list)

/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21
['Vehicle registration plate', 'Traffic sign', 'Car', 'Human body']


In [ ]:
## Create data.yaml for scald-YOLOv4 in dataset folder 
# for invoke if data.yaml, train and valid folder are same level like YOLO model root dir (scaled YOLO)
def create_data_yaml(classes_list, path_to_labelled_raw_dataset, train_foldername, train_image_foldername, valid_foldername, valid_image_foldername):
  #os.chdir(path_to_labelled_raw_dataset)
  os.chdir(path_to_labelled_raw_dataset)

  with open('data.yaml', 'w') as f:
      f.write("train: ../{}/{}".format(train_foldername,train_image_foldername ))
      f.write("\n")
      f.write("val: ../{}/{}".format(valid_foldername, valid_image_foldername))
      f.write("\n")
      f.write("")
      f.write("\n")
      f.write("nc: {}".format(len(classes_list)))
      f.write("\n")
      f.write("names: {}".format(classes_list))

  return f

In [ ]:
## create data.yaml with folder names for invoke(Scaled YOLO) in same folder as YOLO root dir 
# pre:

train_folder_splitted = train_folder.split('/')
name_train =  train_folder_splitted[-1]
print(name_train)

train_image_folder_splitted = path_train_images.split('/')
name_train_images =  train_image_folder_splitted[-1]
print(name_train_images)

valid_folder_splitted = valid_folder.split('/')
name_valid =  valid_folder_splitted[-1]
print(name_valid)

valid_image_folder_splitted = path_valid_images.split('/')
name_valid_images =  valid_image_folder_splitted[-1]
print(name_valid_images)

## change to directory you want data.yaml and call function
os.chdir(path_to_labelled_raw_dataset)
create_data_yaml(classes_txt_list, path_to_labelled_raw_dataset, name_train, name_train_images ,name_valid,name_valid_images)

train
images
valid
images


<_io.TextIOWrapper name='data.yaml' mode='w' encoding='UTF-8'>

# get shuffled image file list

In [ ]:
# input : path labelled src dataset
# output: shuffled image file list
def get_shuffled_imagefile_list(path_to_labelled_raw_dataset):  

    list_all_dataset = os.listdir(path_to_labelled_raw_dataset) 
    list_image_files = []
    list_label_files = []

    for file in os.listdir(path_to_labelled_raw_dataset):
      if file.endswith('.png') or file.endswith('.jpeg') or file.endswith('.jpg') or file.endswith('.gif'):
        list_image_files.append(file)
      if file.endswith('.txt'):
        list_label_files.append(file)
    print('Alphabetic Image list: {}'.format(sorted(list_image_files)))
    print('Alphabetic Labels list: {}'.format(sorted(list_label_files)))

    random.shuffle(list_image_files)
    print('Shuffled Image List: {}'.format(list_image_files))

    return list_image_files

In [ ]:
randomized_image_list = get_shuffled_imagefile_list(path_to_labelled_raw_dataset)

Alphabetic Image list: ['000f269d7fece350.jpg', '002bdd2550665a64.jpg', '002d099996f3f443.jpg', '002f860e692757f7.jpg', '0030164dc0dba5d9.jpg', '0030feaa83a02c0d.jpg', '0039ac4bfb8bd69d.jpg', '00494d0811b9e652.jpg', '004cc310bb32da6a.jpg', '005932632ad92371.jpg', '00670d2aef4f1fcf.jpg', '00765d32b21fe7a4.jpg', '009dc03a884f0bae.jpg', '00a09b822d470896.jpg', '00b5ac570d995133.jpg', '00c462941507d432.jpg', '00cbc1358d338fe0.jpg', '0112fff372a9199f.jpg', '01244840c068d87a.jpg', '01ce2560aff03477.jpg', '021a3f4ad16708b8.jpg', '027348743a9b8374.jpg', '02a01a62fa5a9831.jpg', '02b15a54068319fc.jpg', '030b9726b02097ee.jpg', '033ecc2e897a1ebf.jpg', '0354ed1629a4b1b2.jpg', '03d18376979a3104.jpg', '03d1b4e7ad05beae.jpg', '03fcc4d4897ce9a0.jpg', '0448b2215866ada5.jpg', '047f55a198882d7c.jpg', '04c856a70bb52457.jpg', '04ea4e67d46cd424.jpg', '0500f85f6c7f191d.jpg', '05355bdf2b20ddb1.jpg', '056a66236e64de0c.jpg', '05c52ccdb095f060.jpg', '0647c745934ff80e.jpg', '06501a5ed994c937.jpg', '06861fda7b81fa4

# Create Train,Validation, Test lists depending on defined percentages of dataset

In [ ]:
# creates lists of train-, validation- and testdata of images and textfiles depending on defined percentages on source dataset
# input: path_to_labelled_raw_dataset, numbers_origin_dataset_files, randomized_image_list, percentage_train, percentage_valid, percentage_test
# output: lists images (train, validation and test) and labels/textfiles (train, validation and test)

def create_imagelist_percent_incl_textfiles(path_to_labelled_raw_dataset, numbers_origin_dataset_files, randomized_image_list, percentage_train, percentage_valid, percentage_test):

  #number_files = percentage_of_dataset * numbers_origin_dataset_files/2
  #number_files_rounded = round(number_files)
  #print('Number of files to move rounded {}'.format(number_files_rounded))

  print('Number of files origin Dataset: {}'.format(numbers_origin_dataset_files))
  print('Number of images origin Dataset: {}'.format(numbers_origin_dataset_files/2))


  number_imagefiles_in_data =  len(sorted_dataset_files_list)/2
  number_testfiles  = test_percent * number_imagefiles_in_data
  number_trainfiles = train_percent * number_imagefiles_in_data
  number_validfiles = valid_percent * number_imagefiles_in_data
  rounded_number_testfiles  = round(number_testfiles)
  rounded_number_trainfiles = round(number_trainfiles)
  rounded_number_validfiles = round(number_validfiles)
  full_amount_file_after_round = rounded_number_testfiles + rounded_number_trainfiles + rounded_number_validfiles
  left_files_amount = len(sorted_dataset_files_list)/2 - rounded_number_testfiles - rounded_number_trainfiles - rounded_number_validfiles

  print(number_testfiles)
  print(number_trainfiles)
  print(number_validfiles)
  print('Rounded number of testfiles:{}'.format(rounded_number_testfiles))
  print('Rounded number of training files:{}'.format(rounded_number_trainfiles))
  print('Rounded number of validation files:{}'.format(rounded_number_validfiles))
  print('Full_amount_images_after_round:{}'.format(full_amount_file_after_round*2))
  print('left image files before correction: {}'.format(left_files_amount))
  print('number dataset files before correction: {}'.format(numbers_origin_dataset_files))

  if left_files_amount == -1:
      #numbers_origin_dataset_files = numbers_origin_dataset_files - 2
      rounded_number_testfiles = rounded_number_testfiles - 1
      left_files_amount = len(sorted_dataset_files_list)/2 - rounded_number_testfiles - rounded_number_trainfiles - rounded_number_validfiles
  elif left_files_amount == 1:
      #numbers_origin_dataset_files = numbers_origin_dataset_files + 2
      rounded_number_testfiles = rounded_number_testfiles + 1
      left_files_amount = len(sorted_dataset_files_list)/2 - rounded_number_testfiles - rounded_number_trainfiles - rounded_number_validfiles
  else:
      pass

  print('left files after correction: {}'.format(left_files_amount))
  print('number testfiles images after correction: {}'.format( rounded_number_testfiles))
  print('number dataset files after correction: {}'.format(numbers_origin_dataset_files))

  test_image_file_list_move = randomized_image_list[ : rounded_number_testfiles]
  train_image_file_list_move = randomized_image_list[rounded_number_testfiles : rounded_number_testfiles + rounded_number_trainfiles]
  valid_image_file_list_move = randomized_image_list[rounded_number_testfiles+rounded_number_trainfiles : ]

  print(train_image_file_list_move)
  print(len(train_image_file_list_move))
  print(valid_image_file_list_move)
  print(len(valid_image_file_list_move))
  print(test_image_file_list_move)
  print(len(test_image_file_list_move))

  return train_image_file_list_move, valid_image_file_list_move, test_image_file_list_move


# Get labels list corrsponding to image list of percentages

In [ ]:
## get_labellist_from_imagelist finds textfiles for imagefilenames(list) and saves as list
# input: apth to raw dataset, image list to move
# corresponding textfiles/ labelfiles to image file as list

def get_labellist_from_imagelist(path_to_labelled_src_raw_dataset, images_list):

  file_dict = {}
  list_all_dataset_images = [] #images and textfiles
  image_string_start_list = []  
  labels_file_list = []

  for i in range(len(images_list)):
    #image_file_list_move.append(randomized_image_list[i])
    # split image string at . to get only part in front

    image_string_start = images_list[i].split('.')
    image_string_start_list.append(image_string_start)
    #print(len(image_string_start_list))

    ## create textstring from imagefile and check if textfile in directory
    string_textfile = image_string_start[0] + '.txt'
    #print(string_textfile)
    for file in os.listdir(path_to_labelled_src_raw_dataset):
      if os.path.isfile(os.path.join(path_to_labelled_src_raw_dataset, file)):
        if file == string_textfile:   # image string = 
          #print('labelfile {} corresponding to image file {} '.format(file, images_list[i]))
          labels_file_list.append(file)
          len(labels_file_list)
        else:
          pass
          #print('Image File {} fits not textfile'.format(images_list[i]))
  

  if len(images_list)==len(labels_file_list):
    print('')
    print('ALL labels in dataset and saved in list')
    print(image_string_start_list)
    print(images_list)
    print(labels_file_list)
    print('length image list:     {}'.format(len(images_list)))
    print('lentgth labels list:   {}'.format(len(labels_file_list)))
    
    return labels_file_list

  else:
    print('Label missing for Image')
    assert False

# Move all Images and Textfiles (Train, Valid, Test) to correct/created/corresponding folders

In [ ]:
## move_dataset_files:
# move dataset lists (for images and textfiles/labels) for train, valid and test into correct folders
# Path orgin dataset --> created folder
def move_dataset_files(path_to_labelled_src_raw_dataset, path_image_folder_dest, path_labels_folder_dest, image_list, labels_list):

  for i in range(len(image_list)):
    
    image_file_src_fullpath = path_to_labelled_src_raw_dataset + '/' + image_list[i]
    text_file_src_fullpath = path_to_labelled_src_raw_dataset + '/' + labels_list[i]

    image_file_dest_fullpath = path_image_folder_dest + '/' + image_list[i]
    text_file_dest_fullpath = path_labels_folder_dest + '/' + labels_list[i]


    shutil.move(image_file_src_fullpath, image_file_dest_fullpath)
    shutil.move(text_file_src_fullpath, text_file_dest_fullpath)


# Invoke functions

In [ ]:
### INVOKE FUNCTIONS to move files into correct folders
# invoke image list generator
# outpus is a random organized image list of all images in dataset
randomized_image_list= get_shuffled_imagefile_list(path_to_labelled_raw_dataset)
print(randomized_image_list)

Alphabetic Image list: ['000f269d7fece350.jpg', '002bdd2550665a64.jpg', '002d099996f3f443.jpg', '002f860e692757f7.jpg', '0030164dc0dba5d9.jpg', '0030feaa83a02c0d.jpg', '0039ac4bfb8bd69d.jpg', '00494d0811b9e652.jpg', '004cc310bb32da6a.jpg', '005932632ad92371.jpg', '00670d2aef4f1fcf.jpg', '00765d32b21fe7a4.jpg', '009dc03a884f0bae.jpg', '00a09b822d470896.jpg', '00b5ac570d995133.jpg', '00c462941507d432.jpg', '00cbc1358d338fe0.jpg', '0112fff372a9199f.jpg', '01244840c068d87a.jpg', '01ce2560aff03477.jpg', '021a3f4ad16708b8.jpg', '027348743a9b8374.jpg', '02a01a62fa5a9831.jpg', '02b15a54068319fc.jpg', '030b9726b02097ee.jpg', '033ecc2e897a1ebf.jpg', '0354ed1629a4b1b2.jpg', '03d18376979a3104.jpg', '03d1b4e7ad05beae.jpg', '03fcc4d4897ce9a0.jpg', '0448b2215866ada5.jpg', '047f55a198882d7c.jpg', '04c856a70bb52457.jpg', '04ea4e67d46cd424.jpg', '0500f85f6c7f191d.jpg', '05355bdf2b20ddb1.jpg', '056a66236e64de0c.jpg', '05c52ccdb095f060.jpg', '0647c745934ff80e.jpg', '06501a5ed994c937.jpg', '06861fda7b81fa4

In [ ]:
print('Next function invoke')
# invoke image list generator
list_files_percentages = create_imagelist_percent_incl_textfiles(path_to_labelled_raw_dataset, numbers_origin_dataset_files, randomized_image_list, train_percent, test_percent, valid_percent)

train_image_file_list_move = list_files_percentages[0]
valid_image_file_list_move = list_files_percentages[1]
test_image_file_list_move = list_files_percentages[2]

Next function invoke
Number of files origin Dataset: 1176
Number of images origin Dataset: 588.0
88.20000000000003
411.59999999999997
88.2
Rounded number of testfiles:88
Rounded number of training files:412
Rounded number of validation files:88
Full_amount_images_after_round:1176
left image files before correction: 0.0
number dataset files before correction: 1176
left files after correction: 0.0
number testfiles images after correction: 88
number dataset files after correction: 1176
['b29fa1494534466d.jpg', 'd9659153c0594f95.jpg', '4b763a824d1c62c4.jpg', '2a74702db6eaad09.jpg', '4193f0f2adb1e115.jpg', 'dcadd29eb3117b42.jpg', '6177322e51ba802a.jpg', '2d809fab82a1065a.jpg', '2c1b586b13c1666b.jpg', 'c42618b19cda4dd2.jpg', 'b04254ed4c479cfa.jpg', 'b836089cf8f6a186.jpg', '6f65c53e20426701.jpg', 'dff7b7a341784c05.jpg', '8458411a4e4d8201.jpg', '009dc03a884f0bae.jpg', 'aebe60f28193ef9b.jpg', 'fe531f1bc6187758.jpg', '9874e25925386559.jpg', '38607103ed969efa.jpg', 'f17ab9fabfda1985.jpg', '72a508

In [ ]:
train_text_file_list_move  = get_labellist_from_imagelist(path_to_labelled_raw_dataset, train_image_file_list_move)
valid_text_file_list_move  = get_labellist_from_imagelist(path_to_labelled_raw_dataset, valid_image_file_list_move)
test_text_file_list_move   = get_labellist_from_imagelist(path_to_labelled_raw_dataset, test_image_file_list_move)

print('')
print('After functions')
print('Train image files:   {}'.format(train_image_file_list_move))
print('Train labels files:  {}'.format(train_text_file_list_move))
print('Valid image files:   {}'.format(valid_image_file_list_move))
print('Valid labels files:  {}'.format(valid_text_file_list_move))
print('Test image files:    {}'.format(test_image_file_list_move))
print('Test labels files:   {}'.format(test_text_file_list_move))


ALL labels in dataset and saved in list
[['b29fa1494534466d', 'jpg'], ['d9659153c0594f95', 'jpg'], ['4b763a824d1c62c4', 'jpg'], ['2a74702db6eaad09', 'jpg'], ['4193f0f2adb1e115', 'jpg'], ['dcadd29eb3117b42', 'jpg'], ['6177322e51ba802a', 'jpg'], ['2d809fab82a1065a', 'jpg'], ['2c1b586b13c1666b', 'jpg'], ['c42618b19cda4dd2', 'jpg'], ['b04254ed4c479cfa', 'jpg'], ['b836089cf8f6a186', 'jpg'], ['6f65c53e20426701', 'jpg'], ['dff7b7a341784c05', 'jpg'], ['8458411a4e4d8201', 'jpg'], ['009dc03a884f0bae', 'jpg'], ['aebe60f28193ef9b', 'jpg'], ['fe531f1bc6187758', 'jpg'], ['9874e25925386559', 'jpg'], ['38607103ed969efa', 'jpg'], ['f17ab9fabfda1985', 'jpg'], ['72a50835c7daacaa', 'jpg'], ['8f7344b8b54b9321', 'jpg'], ['0df521fc1e8ee3e7', 'jpg'], ['0647c745934ff80e', 'jpg'], ['a826571d72815d08', 'jpg'], ['badcd10c95dd2eea', 'jpg'], ['a30e019b4669f7a8', 'jpg'], ['9e9fd03e930a6d69', 'jpg'], ['f8e3453eeacec983', 'jpg'], ['de03d3b0d7dd2143', 'jpg'], ['e534d2af3a6b1a34', 'jpg'], ['de1b04ec2b6d8c05', 'jpg'], [

In [ ]:
## invoke moving files into folder structure
# move trainfiles, validfiles, testfiles
move_dataset_files(path_to_labelled_raw_dataset, path_train_images, path_train_labels, train_image_file_list_move, train_text_file_list_move)
move_dataset_files(path_to_labelled_raw_dataset, path_valid_images, path_valid_labels, valid_image_file_list_move, valid_text_file_list_move)
move_dataset_files(path_to_labelled_raw_dataset, path_test_images, path_test_labels, test_image_file_list_move, test_text_file_list_move)

In [ ]:
os.chdir(path_to_labelled_raw_dataset)
file = pathlib.Path(zipfile_full_dataset + '.zip')
if file.exists():
    print ("File exists already")
else:
    print ("File not exist, will be created")

path_zipfile = pathlib.Path(path_to_labelled_raw_dataset + '/' + zipfile_full_dataset + '.zip')
print(path_zipfile)

if pathlib.Path.exists(path_zipfile):
    print('Zipfile already exists, delete first if old')
else:
  shutil.make_archive(zipfile_full_dataset, 'zip', path_to_labelled_raw_dataset)


File not exist, will be created
/content/gdrive/MyDrive/YOLO/DATASET/Vehicle registration plate_Traffic sign_Car_Human body_08_02_21/scaledYOLO_Dataset.zip
